In [1]:
import pandas as pd
import numpy as np

In [2]:
compustat = pd.read_csv("Compustat19612018.csv")
crsp = pd.read_csv("CRSP2007_2018.csv")
linking_table = pd.read_csv("CompustatMergedDatabase.csv")

In [3]:
# We need link gvkey to compustat if compustat.datadate is between linking_table.LINKDT and linking_table.LINKENDDT
linking_table.head()

,gvkey,conm,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
0,1000,A & E PLASTIK PAK INC,P,01,LU,25881,23369,19701113,19780630
1,1001,A & M FOOD SERVICES INC,P,01,LU,10015,6398,19830920,19860731
2,1002,AAI CORP,C,01,LC,10023,22159,19721214,19730605
3,1003,A.A. IMPORTING CO INC,C,01,LU,10031,6672,19831207,19890816
4,1004,AAR CORP,P,01,LU,54594,20000,19720424,E


In [16]:
compustat.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,curcd,act,...,pstkrv,re,revt,sale,seq,tie,txp,txt,costat,datadate_date
0,1000,12/31/1961,1961,INDL,C,D,STD,AE.2,USD,NaN,...,NaN,NaN,0.900,0.900,NaN,NaN,NaN,0.050,I,1961-12-31
1,1000,12/31/1962,1962,INDL,C,D,STD,AE.2,USD,NaN,...,NaN,NaN,1.600,1.600,NaN,NaN,NaN,0.090,I,1962-12-31
2,1000,12/31/1963,1963,INDL,C,D,STD,AE.2,USD,0.408,...,0.0,0.052,1.457,1.457,0.553,NaN,0.000,0.001,I,1963-12-31
3,1000,12/31/1964,1964,INDL,C,D,STD,AE.2,USD,0.718,...,0.0,0.001,2.032,2.032,0.607,NaN,0.007,0.020,I,1964-12-31
4,1000,12/31/1965,1965,INDL,C,D,STD,AE.2,USD,0.725,...,0.0,-0.196,1.688,1.688,0.491,NaN,0.000,-0.103,I,1965-12-31


In [5]:
# function to change E to today()
from datetime import date
def change_LINKENDDT(x):
    if(x=='E'): x = date.today()
    else : pass
    return x
change_LINKENDDT_vectorize = np.vectorize(change_LINKENDDT)

In [6]:
linking_table['LINKENDDT_date'] = pd.to_datetime(change_LINKENDDT_vectorize(linking_table['LINKENDDT']),infer_datetime_format=True)
linking_table['LINKDT_date'] = pd.to_datetime(linking_table['LINKDT'],infer_datetime_format=True)
compustat['datadate_date'] = pd.to_datetime(compustat['datadate'],infer_datetime_format=True)

In [22]:
compustat_linked_merge = compustat.merge(linking_table,how='outer',left_on=['gvkey'],right_on=['gvkey'],validate='many_to_many')

In [35]:
# drop if compustat dates are between effective start and end
compustat_linked_merge_drop = compustat_linked_merge[(compustat_linked_merge.datadate_date >= compustat_linked_merge.LINKDT_date) & (compustat_linked_merge.datadate_date <=  compustat_linked_merge.LINKENDDT_date)]

In [38]:
compustat_linked_merge.shape

(572454, 48)

In [39]:
compustat_linked_merge_drop.shape

(388169, 48)

In [40]:
compustat.shape

(474879, 38)

In [14]:
#df = compustat_linked.drop(compustat_linked[(compustat_linked.datadate >= compustat_linked.LINKDT) & (compustat_linked.datadate <=  compustat_linked.LINKENDDT)].index)

In [15]:
compustat.groupby(['gvkey','datadate']).size().sort_values(ascending=False).head()

gvkey  datadate  
4736   12/31/1970    2
61815  12/31/2008    2
       12/31/2010    2
       12/31/2011    2
       12/31/2012    2
dtype: int64

In [43]:
linking_table['gvkey'].nunique()

25077

In [44]:
compustat['gvkey'].nunique()

33471